### Imports

In [1]:
import requests
import pandas as pd
import json
import time
import os

### Function for API Call to Pandas df

In [2]:
def query(query: str = "", country: str = "cnt:singapore", page: int = 1) -> pd.DataFrame:
    """
    This function takes in the query(species), country with cnt: at the start,
    page as page number. This will output a dataframe from the reqeust of the 
    xeno canto api
    ---
    Args: query:str,country:str, page:str
    ---
    Returns: Dataframe consisting of the data retrieved from the api
    """
    try:
        response = requests.get(
            f"https://xeno-canto.org/api/2/recordings?query={query}{country}&page={page}",
            timeout=5
        )
        if response.status_code != 200:
            print(f"Request failed with status code {response.status_code}")
            return None
        data = response.json()
        if 'recordings' not in data:
            print("No recordings found")
            return None
        recordings = data['recordings']
        df = pd.json_normalize(recordings)
        time.sleep(1)
        
        json_obj = json.loads(response.text)
        num_recordings = json_obj['numRecordings']
        num_species = json_obj['numSpecies']
        page = json_obj['page']
        num_pages = json_obj['numPages']

        print("Number of recordings: ", num_recordings),
        print("Number of species: ", num_species)
        print("Page: ", page)
        print("Number of pages: ", num_pages)
        return df
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [3]:
page = [1,2,3,4,5]
df_sg = pd.DataFrame()

for i  in page:
    df_temp = query(country="cnt:singapore",page=i)
    df_sg = df_sg.append(df_temp)

Number of recordings:  2118
Number of species:  230
Page:  1
Number of pages:  5
Number of recordings:  2118
Number of species:  230
Page:  2
Number of pages:  5
Number of recordings:  2118
Number of species:  230
Page:  3
Number of pages:  5
Number of recordings:  2118
Number of species:  230
Page:  4
Number of pages:  5
Number of recordings:  2118
Number of species:  230
Page:  5
Number of pages:  5


### Meta Data

In [4]:
df_sg['sp'].value_counts()

zeylanicus       62
paradiseus       56
chinensis        51
borealis         34
zanthopygia      34
                 ..
hirundinaceus     1
maculatus         1
mozambica         1
paradisi          1
bergii            1
Name: sp, Length: 203, dtype: int64

In [5]:
df_sg.columns

Index(['id', 'gen', 'sp', 'ssp', 'group', 'en', 'rec', 'cnt', 'loc', 'lat',
       'lng', 'alt', 'type', 'sex', 'stage', 'method', 'url', 'file',
       'file-name', 'lic', 'q', 'length', 'time', 'date', 'uploaded', 'also',
       'rmk', 'bird-seen', 'animal-seen', 'playback-used', 'temp', 'regnr',
       'auto', 'dvc', 'mic', 'smp', 'sono.small', 'sono.med', 'sono.large',
       'sono.full', 'osci.small', 'osci.med', 'osci.large'],
      dtype='object')

   
This is the schema obtained from the website, which is missing ['sono.small', 'sono.med', 'sono.large','sono.full', 'osci.small', 'osci.med', 'osci.large']

* id: the catalogue number of the recording on xeno-canto
* gen: the generic name of the species
* sp: the specific name (epithet) of the species
* ssp: the subspecies name (subspecific epithet)
* group: the group to which the species belongs (birds, grasshoppers)
* en: the English name of the species
* rec: the name of the recordist
* cnt: the country where the recording was made
* loc: the name of the locality
* lat: the latitude of the recording in decimal coordinates
* lng: the longitude of the recording in decimal coordinates
* type: the sound type of the recording (combining both predefined terms such as 'call' or 'song' and additional free text options)
* sex: the sex of the animal
* stage: the life stage of the animal (adult, juvenile, etc.)
* method: the recording method (field recording, in the hand, etc.)
* url: the URL specifying the details of this recording
* file: the URL to the audio file
* file-name: the original file name of the audio file
* sono: an object with the urls to the four versions of sonograms
* osci: an object with the urls to the three versions of oscillograms
* lic: the URL describing the license of this recording
* q: the current quality rating for the recording
* length: the length of the recording in minutes
* time: the time of day that the recording was made
* date: the date that the recording was made
* uploaded: the date that the recording was uploaded to xeno-canto
* also: an array with the identified background species in the recording
* rmk: additional remarks by the recordist
* bird-seen: despite the field name (which was kept to ensure backwards compatibility), this field indicates whether the recorded animal was seen
* animal-seen: was the recorded animal seen?
* playback-used: was playback used to lure the animal?
* temperature: temperature during recording (applicable to specific groups only)
* regnr: registration number of specimen (when collected)
* auto: automatic (non-supervised) recording?
* dvc: recording device used
* mic: microphone used
* smp: sample rate

In [6]:
df_sg['file']

0      https://xeno-canto.org/578938/download
1      https://xeno-canto.org/728917/download
2      https://xeno-canto.org/689265/download
3      https://xeno-canto.org/677303/download
4      https://xeno-canto.org/550149/download
                        ...                  
113    https://xeno-canto.org/615496/download
114    https://xeno-canto.org/610700/download
115    https://xeno-canto.org/574849/download
116    https://xeno-canto.org/689841/download
117    https://xeno-canto.org/689840/download
Name: file, Length: 2118, dtype: object

In [7]:
os.chdir('..')
os.getcwd()

'c:\\Users\\Justin\\Desktop\\AI CV\\Sound Duck\\SoundDuck'

In [8]:
birds_of_interest = ['paradiseus', 'chinensis', 'zanthopygia']
df = df_sg.loc[df_sg['sp'].isin(birds_of_interest)].copy()

In [9]:
df.drop(df.loc[df['file'] == ''].index, inplace=True)

In [10]:
df['file']

0      https://xeno-canto.org/578938/download
162    https://xeno-canto.org/773797/download
163    https://xeno-canto.org/756521/download
164    https://xeno-canto.org/756425/download
165     https://xeno-canto.org/40042/download
                        ...                  
335    https://xeno-canto.org/341563/download
336     https://xeno-canto.org/90034/download
337    https://xeno-canto.org/637962/download
338    https://xeno-canto.org/578798/download
339    https://xeno-canto.org/185284/download
Name: file, Length: 97, dtype: object

### Function to download Audio Files from a Pandas DataFrame

In [11]:
def download_files(df):
    """
    Downloads audio files from URLs specified in a Pandas DataFrame,
    saves them to a local 'data' folder, and appends the file path as
    a new column to the DataFrame.
    ---
    Args:
        df: Pandas DataFrame with 'file' and 'file-name' columns specifying
            the URLs and file names of the audio files to be downloaded.
    ---
    Returns:
        df: Pandas DataFrame with an additional 'file-path' column that
            contains the local file paths of the downloaded audio files.
    """
    for i, row in df.iterrows():
        url = row['file']
        file_name = row['file-name']
        response = requests.get(url)
        file_path = os.path.join('data', file_name)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"File '{file_name}' downloaded to 'data' folder.")
        df.loc[i, 'file-path'] = file_path
        
    return df

In [12]:
download_files(df)

File 'XC578938-Quail, King_2019-12-28_0757_SG_Harvest-Link.mp3' downloaded to 'data' folder.
File 'XC773797-SpottedDove_SungeiBuloh_230106_3358_edited.mp3' downloaded to 'data' folder.
File 'XC756521-Dove_Spotted_2022-01-27_0750_SG_SBG.mp3' downloaded to 'data' folder.
File 'XC756425-Dove_Spotted_2021-09-01_0814_SG_SBTB.mp3' downloaded to 'data' folder.
File 'Spotted_Dove_sang_Singapore_botaniska_tradgard_2009-07-07.mp3' downloaded to 'data' folder.
File 'XC756522-Dove_Spotted_2022-02-02_1001_SG_PasirPanjang.mp3' downloaded to 'data' folder.
File 'XC488677-Spotted_Dove_02.mp3' downloaded to 'data' folder.
File 'XC723384-audiomass-output.mp3' downloaded to 'data' folder.
File 'XC576681-Drongo, Greater Racket-tailed_2020-07-06_0932_SG_CCNR.mp3' downloaded to 'data' folder.
File 'XC576680-Drongo, Greater Racket-tailed_2020-07-06_0724_SG_CCNR.mp3' downloaded to 'data' folder.
File 'XC488686-Racket_tailed_Drongo_02.mp3' downloaded to 'data' folder.
File 'XC402975-Drongo, Greater Racket-tail

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,mic,smp,sono.small,sono.med,sono.large,sono.full,osci.small,osci.med,osci.large,file-path
0,578938,Excalfactoria,chinensis,,birds,King Quail,Okamoto Keita Sin,Singapore,"Kranji Marshes, Singapore",1.4202,...,,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,"data\XC578938-Quail, King_2019-12-28_0757_SG_H..."
162,773797,Spilopelia,chinensis,,birds,Spotted Dove,Bram Piot,Singapore,Sungei Buloh Wetland Reserve,1.448,...,,44100,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,data\XC773797-SpottedDove_SungeiBuloh_230106_3...
163,756521,Spilopelia,chinensis,,birds,Spotted Dove,Okamoto Keita Sin,Singapore,"Singapore Botanic Gardens, Singapore",1.3139,...,Sennheiser MKE600,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,data\XC756521-Dove_Spotted_2022-01-27_0750_SG_...
164,756425,Spilopelia,chinensis,,birds,Spotted Dove,Okamoto Keita Sin,Singapore,"Satay by the Bay, Singapore",1.2824,...,Sennheiser MKE600,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,data\XC756425-Dove_Spotted_2021-09-01_0814_SG_...
165,40042,Spilopelia,chinensis,,birds,Spotted Dove,Patrik Åberg,Singapore,Singapore Botanic Gardens,1.313095,...,,44100,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,data\Spotted_Dove_sang_Singapore_botaniska_tra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,341563,Ficedula,zanthopygia,,birds,Yellow-rumped Flycatcher,Kim Chuah Lim,Singapore,Singapore Zoo,1.4043,...,,44100,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,data\XC341563-Yellow-rumped Flycatcher @ Zoo_1...
336,90034,Ficedula,zanthopygia,,birds,Yellow-rumped Flycatcher,Yong Ding Li,Singapore,"Bidadari Park, Singapore",1.341,...,,44100,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,data\Yellow-rumped Flycatcher_bidadari.mp3
337,637962,Ficedula,zanthopygia,,birds,Yellow-rumped Flycatcher,Kim Chu

In [13]:
df['file-path']

0      data\XC578938-Quail, King_2019-12-28_0757_SG_H...
162    data\XC773797-SpottedDove_SungeiBuloh_230106_3...
163    data\XC756521-Dove_Spotted_2022-01-27_0750_SG_...
164    data\XC756425-Dove_Spotted_2021-09-01_0814_SG_...
165    data\Spotted_Dove_sang_Singapore_botaniska_tra...
                             ...                        
335    data\XC341563-Yellow-rumped Flycatcher @ Zoo_1...
336           data\Yellow-rumped Flycatcher_bidadari.mp3
337    data\XC637962-Yellow-rumped Flycatcher @ DFNP ...
338    data\XC578798-Flycatcher, Yellow-rumped_2019-1...
339    data\XC185284-Yellow-rumped Flycatcher_Singapo...
Name: file-path, Length: 97, dtype: object

In [15]:
file_path = os.path.join(os.getcwd(), "data/csv", "my_data.csv")
df.to_csv(file_path, index=False)

In [16]:
df = pd.read_csv(file_path)
df

,id,gen,sp,ssp,group,en,rec,cnt,loc,lat,...,mic,smp,sono.small,sono.med,sono.large,sono.full,osci.small,osci.med,osci.large,file-path
0,578938,Excalfactoria,chinensis,NaN,birds,King Quail,Okamoto Keita Sin,Singapore,"Kranji Marshes, Singapore",1.420200,...,NaN,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,"data\XC578938-Quail, King_2019-12-28_0757_SG_H..."
1,773797,Spilopelia,chinensis,NaN,birds,Spotted Dove,Bram Piot,Singapore,Sungei Buloh Wetland Reserve,1.448000,...,NaN,44100,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/ff...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,//xeno-canto.org/sounds/uploaded/RGSEIIJGHA/wa...,data\XC773797-SpottedDove_SungeiBuloh_230106_3...
2,756521,Spilopelia,chinensis,NaN,birds,Spotted Dove,Okamoto Keita Sin,Singapore,"Singapore Botanic Gardens, Singapore",1.313900,...,Sennheiser MKE600,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,data\XC756521-Dove_Spotted_2022-01-27_0750_SG_...
3,756425,Spilopelia,chinensis,NaN,birds,Spotted Dove,Okamoto Keita Sin,Singapore,"Satay by the Bay, Singapore",1.282400,...,Sennheiser MKE600,48000,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/ff...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,//xeno-canto.org/sounds/uploaded/ZPYEHLWXXX/wa...,data\XC756425-Dove_Spotted_2021-09-01_0814_SG_...
4,40042,Spilopelia,chinensis,NaN,birds,Spotted Dove,Patrik Åberg,Singapore,Singapore Botanic Gardens,1.313095,...,NaN,44100,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/ff...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,//xeno-canto.org/sounds/uploaded/BPSDQEOJWG/wa...,data\Spotted_Dove_sang_Singapore_botaniska_tra...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,341563,Ficedula,zanthopygia,NaN,birds,Yellow-rumped Flycatcher,Kim Chuah Lim,Singapore,Singapore Zoo,1.404300,...,NaN,44100,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/ff...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,//xeno-canto.org/sounds/uploaded/FRBMXJHOSE/wa...,data\XC341563-Yellow-rumped Flycatcher @ Zoo_1...
93,90034,Ficedula,zanthopygia,NaN,birds,Yellow-rumped Flycatcher,Yong Ding Li,Singapore,"Bidadari Park, Singapore",1.341000,...,NaN,44100,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/ff...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,//xeno-canto.org/sounds/uploaded/NWTKEXMKCA/wa...,data\Yellow-rumped Flycatcher_bidadari.mp3
94,637962,Ficedula,zanthopygia,